In [141]:
import importlib
import pydicom as dicom

import data
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import matplotlib.pylab as plt
from PIL import Image
import numpy as np
import cv2
importlib.reload(data)

<module 'data' from '/Users/amolmenon/Documents/Spring 2023/CMSC472/cmsc472_medicaldiagnosis/data.py'>

In [142]:
all_data = data.load_datasets()

train_set = all_data['lung']['train']
val_set = all_data['lung']['val']

In [143]:
batch_size = 32
learning_rate = 0.00025
epochs = 300

In [144]:
train_dataloader = DataLoader(train_set, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

In [145]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

#         self.w1 = nn.Linear(256*256, 7)
#         self.relu = nn.ReLU()
#         self.w2 = nn.Linear(7, 1)
#         self.sigmoid = nn.Sigmoid()

    def forward(self, x):     

        z1 = self.w1(x)
        a1 = self.relu(z1)
        z2 = self.w2(a1)
        a2 = self.sigmoid(z2)

        out = a2
        return out

In [146]:
model = Net()  
optim = torch.optim.SGD(model.parameters(), lr = learning_rate) 

In [149]:
train_loss_over_epochs = []
val_loss_over_epochs = []


for epoch in range(epochs):

    #Training phase
    model.train()  #Setting the model to train phase
    train_loss = []
    val_loss = []
    train_acc = 0.
    val_acc = 0.
    
    for idx, batch in enumerate(train_dataloader):
        ## TODO: fetch inputs and labels from batch
        ## TODO: pass inputs to model
        x = batch[0]
        y = batch[1]

        optim.zero_grad()

        out = model(x)

        loss = loss_function(out, y.float().unsqueeze(1))
        loss.backward()
        optim.step()


        ## TODO: call loss function
        ## TODO: call backward
        ## TODO: step optimizer

        ## TODO: compute accuracy and track accuracy and loss
        y_pred = torch.round(out)

        train_acc = torch.sum(torch.square(torch.sub(y, y_pred))).item() / (y.size(dim=0))
        train_acc = (torch.squeeze(y_pred).eq(y)).sum().item() / y.size(dim=0)

        train_loss.append(loss.item())

        ## IMPORTANT: don't forget to call zero_grad on the optimizer between batches!!!

    #Validation phase
    model.eval()  #Setting the model to eval mode, hence making it deterministic.
    for idx, batch in enumerate(val_dataloader):
        with torch.no_grad():   #Does not calulate the graidents, as in val phase its not needed. Saves on memory.
            ## TODO: similar to training, but no need to call backward, step, or zero_grad
            x = batch['image']
            y = batch['label']
            out = model.forward(x)
            loss = loss_function(out, y.float().unsqueeze(1))

            y_pred = torch.round(out)
            val_acc = (torch.squeeze(y_pred).eq(y)).sum().item() / y.size(dim=0)
            val_loss.append(loss.item())
    train_loss_over_epochs.append(np.mean(train_loss))
    val_loss_over_epochs.append(np.mean(val_loss))
    ## DO NOT change the next two lines
    if epoch%100==0:
        print("Epoch : {}, Train loss: {} , Train Acc: {}, Val loss: {}, Val acc: {}".format(epoch, np.mean(train_loss), train_acc, np.mean(val_loss), val_acc))

torch.Size([16, 256, 256])


TypeError: list indices must be integers or slices, not str